In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
df = pd.read_pickle('FT-data-DSpace/data_agreggated.pickle')
df = df[['Date', 'Meeting Number', 'Clean']]

In [20]:
df['Sentences'] = df['Clean'].apply(lambda x: ' '.join(x))
df.sample(2)

,Date,Meeting Number,Clean,Sentences
692,2016-01-26T13:00:00,44. møde,"[mød, åbn, skul, nog, vær, kom, tvivl, altså, ...",mød åbn skul nog vær kom tvivl altså mød nr ik...
143,2010-12-15T09:00:00,35. møde,"[mød, åbn, dag, følg, anmeld, beskæftigelsesmi...",mød åbn dag følg anmeld beskæftigelsesminist i...


In [26]:
tvec = TfidfVectorizer(min_df=.0025, max_df=.1)
x = tvec.fit_transform(df['Sentences'])

In [32]:
x[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [36]:
tvec.get_feature_names()

['0forslag',
 '0lovforslag',
 '130j',
 '180grad',
 '24syv',
 '24tim',
 '2a',
 'a1',
 'a4',
 'a6',
 'aa',
 'aaa',
 'aab',
 'aabenraa',
 'aaby',
 'aabybro',
 'aag',
 'aagaard',
 'aaja',
 'aalborgmodel',
 'aalborgområd',
 'aamund',
 'aarhusborgmest',
 'aarhusbug',
 'aarhusiansk',
 'aarhusmodel',
 'aarhusområd',
 'aas',
 'ab',
 'abadi',
 'abba',
 'abc',
 'abdalla',
 'abdel',
 'abdul',
 'abdullah',
 'abe',
 'abekastning',
 'aben',
 'aber',
 'abespil',
 'abild',
 'abildgaard',
 'abkhasi',
 'abnorm',
 'abon',
 'abonnement',
 'abonnementskort',
 'abonnementsordning',
 'abonnent',
 'abort',
 'abortankenævn',
 'abortsøg',
 'abrupt',
 'abs',
 'absalon',
 'absorb',
 'abstinens',
 'abstrak',
 'abstraktion',
 'abstraktionsniveau',
 'absurdit',
 'absurditet',
 'abt',
 'abu',
 'ac',
 'acceler',
 'acceleration',
 'accent',
 'acceptabl',
 'acces',
 'accompli',
 'accord',
 'accordingly',
 'ace',
 'acontobeløb',
 'acontoerstatning',
 'acontoskat',
 'acontoudbetaling',
 'acquis',
 'act',
 'action',
 'adam'